[View in Colaboratory](https://colab.research.google.com/github/syedmujahedalih/ForskLabs_DL/blob/master/CNN_Pokemon_Classify.ipynb)

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
#!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
#!apt-get update -qq 2>&1 > /dev/null
#!apt-get -y install -qq google-drive-ocamlfuse fuse
!wget https://launchpad.net/~alessandro-strada/+archive/ubuntu/google-drive-ocamlfuse-beta/+build/15331130/+files/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!dpkg -i google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!apt-get install -f
!apt-get -y install -qq fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


Redirecting output to ‘wget-log.1’.
(Reading database ... 19845 files and directories currently installed.)
Preparing to unpack google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.0-0ubuntu1) over (0.7.0-0ubuntu1) ...
Setting up google-drive-ocamlfuse (0.7.0-0ubuntu1) ...
Reading package lists... Done
Building dependency tree       
Reading state information... Done
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.


KeyboardInterrupt: ignored

In [9]:
!mkdir -p drive
!google-drive-ocamlfuse drive

/bin/bash: google-drive-ocamlfuse: command not found


In [0]:
!pip install -q keras

In [1]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

Using TensorFlow backend.


In [0]:
classifier = Sequential()

In [0]:
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))

In [0]:
#Pooling

classifier.add(MaxPooling2D(pool_size = (2, 2)))


In [0]:
# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

In [0]:
# Step 3 - Flattening
classifier.add(Flatten())

In [0]:
# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 3, activation = 'softmax'))

In [0]:
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [12]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# Note that the test data shouldn’t be augmented!
test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('drive/dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('drive/dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 445 images belonging to 3 classes.
Found 238 images belonging to 3 classes.


In [0]:
classifier.fit_generator(training_set,
                         steps_per_epoch = 100,
                         epochs = 5,
                         validation_data = test_set,
                         validation_steps = 100)

Epoch 1/5
  8/100 [=>............................] - ETA: 2:35 - loss: 1.1741 - acc: 0.4609

/usr/local/lib/python2.7/dist-packages/PIL/Image.py:872: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


100/100 [==============================] - 256s 3s/step - loss: 0.3561 - acc: 0.8608 - val_loss: 0.2240 - val_acc: 0.9249
Epoch 2/5
 99/100 [============================>.] - ETA: 1s - loss: 0.0963 - acc: 0.9621